In [40]:
import sys
import os
import torch
import csv
from fairseq.models.bart import BARTModel
from fairseq.data.data_utils import collate_tokens

In [2]:
bart = BARTModel.from_pretrained(
    'checkpoint/',
    checkpoint_file='checkpoint_best.pt',
    data_name_or_path='temp'
)


| [source] dictionary: 50264 types
| [target] dictionary: 50264 types


In [66]:
bart.eval()
bart.half()

BARTHubInterface(
  (model): BARTModel(
    (encoder): TransformerEncoder(
      (embed_tokens): Embedding(50264, 1024, padding_idx=1)
      (embed_positions): LearnedPositionalEmbedding(1026, 1024, padding_idx=1)
      (layers): ModuleList(
        (0): TransformerEncoderLayer(
          (self_attn): MultiheadAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
          (final_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        )
        (1): TransformerEnc

In [7]:
with open('./temp/val.source.tiny') as source:
    slines = [line.strip() for line in source.readlines()]

In [67]:
batch = collate_tokens([bart.encode(s) for s in slines], pad_idx=1)

In [68]:
b_t = bart.extract_features(batch)
x_t = [bart.extract_features(x).squeeze(0) for x in batch]

RuntimeError: _th_index_select not supported on CPUType for Half

In [61]:
b_t.shape, torch.stack(x_t).shape

(torch.Size([10, 53, 1024]), torch.Size([10, 53, 1024]))

In [65]:
b_t.transpose(2,1) == b_t.transpose(1,2)

tensor([[[True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True],
         ...,
         [True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True]],

        [[True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True],
         ...,
         [True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True]],

        [[True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True],
         ...,
         [True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True]],

In [13]:
test = [bart.encode(t) for t in slines]
print(test)

[tensor([    0,  1185,   128,   548,  1747,  2312,     7,  8286,     5,  3556,
            7, 45075,   479, 26825,  2156, 11644,  2092,   137,    47,  2156,
         1420,    47,    10,   265,  1886,  2156,     8,   161,  2156, 45518,
          520,    47,  4883,  1207,  6000, 29384,  2156,   486,    42,   346,
         2156,    38,   128,   548,   300,    10,   633,   904,    13,    47,
          479, 12801,     2]), tensor([    0,   133,  6950,    16,   888,    10,  3065,  8380,  2156,     8,
           24,    34,    95,   554,     7, 22451,   479,     2]), tensor([    0,  1185,   465,    10, 15061,    11,    86,  3051,   149,     5,
         1076, 21242,   479,   370,  2914,    24,     7,   465,  2512,    15,
           10,  4204,  2103,    19, 15535,  9571,    15,    10,  3428,   220,
            7,    47,   479,     2]), tensor([    0,  2709,   107,    11,   110,  2719,     5,   276, 34004,  2048,
         2092,    11,   110,  7416,  2156,    47,    32,   205,   964,   479,
      

In [24]:
torch.stack((test[0][:18],test[1]))

tensor([[    0,  1185,   128,   548,  1747,  2312,     7,  8286,     5,  3556,
             7, 45075,   479, 26825,  2156, 11644,  2092,   137],
        [    0,   133,  6950,    16,   888,    10,  3065,  8380,  2156,     8,
            24,    34,    95,   554,     7, 22451,   479,     2]])

In [26]:
first, second = test[0][:18], test[1]
both = torch.stack((first, second))

In [38]:
f_t = bart.extract_features(first)
b_t = bart.extract_features(both)[0].unsqueeze(0)

In [39]:
f_t == b_t

tensor([[[False, False, False,  ..., False, False, False],
         [False, False, False,  ..., False, False, False],
         [False, False, False,  ..., False, False, False],
         ...,
         [False, False, False,  ..., False, False, False],
         [False, False, False,  ..., False, False, False],
         [False, False, False,  ..., False, False, False]]])

In [21]:
bart.model.encoder.embed_tokens.num_embeddings

50264

In [23]:
bart.extract_features(

In [24]:
x(test)

tensor([[[ 0.4731, -1.0730, -0.7140,  ..., -0.4472, -0.3503,  0.0702],
         [ 0.4265, -1.2010, -0.7244,  ..., -0.5961, -0.5043,  0.0364],
         [-0.6483, -0.5229,  0.1404,  ...,  0.7894,  0.4436,  0.2751],
         ...,
         [-0.2019,  1.3110, -0.4670,  ..., -0.0142,  0.0143, -0.0261],
         [-0.6845,  0.5013, -0.5826,  ...,  0.1284, -0.0781,  0.0097],
         [-0.5286,  1.5292, -0.7406,  ..., -0.6144,  0.1326,  0.2708]]],
       grad_fn=<TransposeBackward0>)

In [32]:
bart.model.encoder.dictionary

In [54]:
print(bart.encode(slines))

TypeError: expected string or buffer

In [39]:
from torch.utils.data import DataLoader, IterableDataset

In [41]:
mydata = IterableDataset()

In [51]:
fields = ["context", "generated", "gold", "label"]
fin = open("disc_train/relevance/disc_train.txt.tsv", newline="")
tsv_reader = csv.DictReader(fin, delimiter='\t', fieldnames=fields)
for line in tsv_reader:
    print(line[fields[0]])
    break

An ordinary bus driver discovers that his bus is magic - it can go anywhere or anywhen , but only if there 's a diverse enough group of children are in it . <EOT> </s>


ValueError: I/O operation on closed file.

In [78]:
t = torch.Tensor([    0,  4688,  7945,  2353,  1393, 28411,    14,    39,  2353,    16,
         8375,   111,    24,    64,   213,  4558,    50,   143, 14746,  2156,
           53,   129,   114,    89,   128,    29,    10,  5544,   615,   333,
            9,   408,    32,    11,    24,   479, 28696,   717,  3293, 15698,
        49703,    29, 15698,     2])
t2 = t = torch.Tensor([    0,  4688,  7945,  2353,  1393, 28411,    14,    39,  2353,    16,
         8375,   111,    24,    64,   213,  4558,    50,   143, 14746,  2156,
           53,   129,   114,    89,   128,    29,    10,  5544,   615,   333,
            9,   408])

In [83]:
t.size()

torch.Size([32])

In [67]:
from torch.nn.utils.rnn import pad_sequence

In [84]:
pad_sequence([t,t2]).size()

torch.Size([32, 2])